In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy
from collections import defaultdict
import xml.etree.ElementTree as ET
import os
import numpy as np
import random
import copy
import pandas as pd
import itertools
import xmltodict
from tqdm import tqdm

def group_by_frame(predicted_boxes):
    """Group the detected boxes by frame_id as a dictionary"""
    predicted_boxes.sort(key=lambda x: x[0])
    predicted_boxes = itertools.groupby(predicted_boxes, key=lambda x: x[0])
    predicted_boxes = {k: list(v) for k, v in predicted_boxes}
    return predicted_boxes

In [2]:
def load_from_xml(path):
    """

    :param path: path file

    :return: dict[frame_num] = [[x1, y1, x2, y2]]
    """
    
    with open(path) as f:
        tracks = xmltodict.parse(f.read())['annotations']['track']

    gt = defaultdict(list)
    num_iter = 0
    for track in tracks:
        label = track['@label']
        boxes = track['box']
        for box in boxes:
            if label == 'car':
                frame=int(box['@frame'])
                frame = f'f_{frame}'
                gt[frame].append(
                    [float(box['@xtl']),
                    float(box['@ytl']),
                    float(box['@xbr']),
                    float(box['@ybr'])]
                )
                num_iter += 1

            else:
                continue

    return gt, num_iter

In [3]:
def load_from_txt(path):
    """
    :param path: path file

    :return: list = [[frame,x1, y1, x2, y2, conf]]
    """
    frame_list = []
    with open(path) as f:
        lines = f.readlines()
   
    for l in lines:
        ll = l.split(',')
        frame = f'f_{int(ll[0]) - 1}'
        frame_list.append([frame,float(ll[2]), float(ll[3]),
                                            float(ll[2]) + float(ll[4]), float(ll[3])+float(ll[5]),ll[6]])
        


    return frame_list

In [4]:
def generate_noisy_boxes(gt_boxes, del_prob,gen_prob, mean, std,frame_shape=[1080, 1920]):
    """
    :gt_boxes: ground truth bounding boxes dict
    :del_prob: probability to delete bounding boxes
    :gen_prob: probability to generate bounding boxes
    :return: list with the noisy bounding boxes list = [[frame,x1, y1, x2, y2]]
    """
    noisy_bboxes = []
    gt_total = 0
    for frame,bboxes in gt_boxes.items():
        for bbox in bboxes:
            gt_total += 1
            if np.random.random() > del_prob:
                xtl, ytl, xbr, ybr = bbox
                noise = np.random.normal(mean,std,4)
                noisy_bboxes.append([frame,xtl+noise[0], ytl+noise[1], xbr+noise[2], ybr+noise[3]])
                w = xbr - xtl
                h = ybr - ytl

        if np.random.random() <= gen_prob:
            x = np.random.randint(w, frame_shape[1]-w)
            y = np.random.randint(h, frame_shape[0]-h)
            noisy_bboxes.append([frame,x-w/2, y-w/2, x+w/2, y+w/2])


    return noisy_bboxes


In [5]:
def iou(box1, box2):
    if len(box1) > 4:
        box1=box1[:4]
    x11, y11, x12, y12 = box1
    x21, y21, x22, y22 = box2
    xA = np.maximum(x11, x21)
    yA = np.maximum(y11, y21)
    xB = np.minimum(x12, x22)
    yB = np.minimum(y12, y22)
    interArea = np.maximum(0, xB - xA + 1) * np.maximum(0, yB - yA + 1)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / np.float64(boxAArea + boxBArea - interArea)
    return iou

In [6]:
def mean_AP_Pascal_VOC(gt_boxes,N_gt,predicted_boxes,iou_th):
    """
    :gt_boxes: ground truth bounding boxes dict
    :N_gt: Total of ground truth bounding boxes
    :predicted_boxes: predicted bounding boxes
    :return: mean IOU, average precision
    """
    mIOU = 0
    tp = np.zeros(len(predicted_boxes))
    fp = np.zeros(len(predicted_boxes))
    gt_detected = copy.deepcopy(gt_boxes)

    mIOU_frame = {}
    for i in range(len(predicted_boxes)):
        frame = predicted_boxes[i][0]
        predicted = predicted_boxes[i][1:5]
        gt = gt_detected[frame]
        iou_score = []
        if len(gt) != 0:
            for b in range(len(gt)):
                iou_score.append(iou(gt[b],predicted))
            id = np.argmax(iou_score)
            max_iou = iou_score[id]
            mIOU += max_iou
            # Save max iou for each frame
            if frame not in mIOU_frame:
                mIOU_frame[frame] = []
            mIOU_frame[frame].append(max_iou)

            if max_iou >= iou_th:
                if len(gt_detected[frame][id]) == 4:
                    gt_detected[frame][id].append(True)
                    tp[i] = 1
                else:
                    fp[i] = 1
            else:
                fp[i] = 1

    tp = np.cumsum(tp)
    fp = np.cumsum(fp)

    recall = tp/ np.float64(N_gt)
    precision = tp/np.maximum(tp + fp, np.finfo(np.float64).eps)

    ap = 0.0
    for t in np.arange(0.0, 1.1, 0.1):
        if np.sum(recall >= t) == 0:
            p = 0
        else:
            p = np.max(precision[recall >= t])
        ap = ap + p / 11.0

    return mIOU/len(predicted_boxes), mIOU_frame, ap

In [7]:
def mean_IoU_restricted(gt_boxes, predicted_boxes):
    """
    :gt_boxes: ground truth bounding boxes dict
    :predicted_boxes: predicted bounding boxes
    :return: mean IOU
    """
    mIOU = 0
    count = 0
    mIOU_frame = {}
    used_pred_boxes = set()

    for gt in gt_boxes:
        for box in gt_boxes[gt]:
            iou_score = []
            for pred in predicted_boxes:
                if pred[0] == gt and tuple(pred[1:5]) not in used_pred_boxes:
                    iou_score.append(iou(box,pred[1:5]))
                else:
                    iou_score.append(0)
            if iou_score:
                id = np.argmax(iou_score)
                max_iou = iou_score[id]
                used_pred_boxes.add(tuple(predicted_boxes[id][1:5]))
                mIOU += max_iou
                count = count + 1

                # Save max iou for each frame
                if gt not in mIOU_frame:
                    mIOU_frame[gt] = []
                mIOU_frame[gt].append(max_iou)
            

    return mIOU/count, mIOU_frame

def mean_IoU_nonrestricted(gt_boxes, predicted_boxes):
    """
    :gt_boxes: ground truth bounding boxes dict
    :predicted_boxes: predicted bounding boxes
    :return: mean IOU
    """
    mIOU = 0
    count = 0
    mIOU_frame = {}

    for gt in gt_boxes:
        for box in gt_boxes[gt]:
            iou_score = []
            for pred in predicted_boxes:
                if pred[0] == gt:
                    iou_score.append(iou(box,pred[1:5]))
                else:
                    iou_score.append(0)
            if iou_score:
                id = np.argmax(iou_score)
                max_iou = iou_score[id]
                mIOU += max_iou
                count = count + 1

                # Save max iou for each frame
                if gt not in mIOU_frame:
                    mIOU_frame[gt] = []
                mIOU_frame[gt].append(max_iou)
            

    return mIOU/count, mIOU_frame




In [8]:
gt_boxes,total = load_from_xml('../dataset/ai_challenge_s03_c010-full_annotation.xml')

maskrcnn = '../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt'
ssd = '../dataset/AICity_data/train/S03/c010/det/det_ssd512.txt'
yolo = '../dataset/AICity_data/train/S03/c010/det/det_yolo3.txt' 

paths = [maskrcnn,yolo,ssd]
for path in paths:
    boxes = load_from_txt(path)
    # Sort by confidence
    boxes.sort(key=lambda x:x[-1], reverse=True)
    

    print(path.split('/')[-1],':')
    mIOU, mIOU_frame = mean_IoU_restricted(gt_boxes,boxes)
    print(f'mIOU restricted: {mIOU}')
    mIOU, mIOU_frame = mean_IoU_nonrestricted(gt_boxes,boxes)
    print(f'mIOU nonrestricted: {mIOU}')


det_mask_rcnn.txt :
mIOU restricted: 0.4930454541180719
mIOU nonrestricted: 0.520667677531192
det_yolo3.txt :
mIOU restricted: 0.31669758812721704
mIOU nonrestricted: 0.45917993364936016
det_ssd512.txt :
mIOU restricted: 0.2665120012610383
mIOU nonrestricted: 0.386826368938464


In [19]:
def compute_confidences_ap(gt_boxes,N_gt,predicted_boxes,N=10,iou_th = 0.5):
    """ 
    Randomly generates the order of the bounding boxes to calculate the average precision (N times). 
    Average values will be returned.
    """
    ap_scores = []
    
    for i in tqdm(range(N)):
        random.shuffle(predicted_boxes)
        _,_, ap = mean_AP_Pascal_VOC(gt_boxes,N_gt,predicted_boxes,iou_th)
        ap_scores.append(ap)
        mIOU, _ = mean_IoU_nonrestricted(gt_boxes,predicted_boxes)
    return sum(ap_scores)/len(ap_scores),mIOU




In [20]:
import multiprocessing as mp

def compute_confidences_ap(gt_boxes, N_gt, predicted_boxes, N=10, iou_th=0.5):
    ap_scores = []

    for i in tqdm(range(N)):
        random.shuffle(predicted_boxes)
        _,_, ap = mean_AP_Pascal_VOC(gt_boxes,N_gt,predicted_boxes,iou_th)
        ap_scores.append(ap)
        mIOU, _ = mean_IoU_nonrestricted(gt_boxes,predicted_boxes)
    return sum(ap_scores)/len(ap_scores),mIOU

def compute_ap_score(q, gt_boxes, N_gt, predicted_boxes, iou_th):
    random.shuffle(predicted_boxes)
    mIOU, _, ap = mean_AP_Pascal_VOC(gt_boxes, N_gt, predicted_boxes, iou_th)
    q.put(ap)

def parallel_compute_confidences_ap(gt_boxes, N_gt, predicted_boxes, N=10, iou_th=0.5):
    ap_scores = []

    with mp.Pool() as pool:
        q = mp.Queue()
        for i in range(N):
            pool.apply_async(compute_ap_score, args=(q, gt_boxes, N_gt, predicted_boxes, iou_th))

        pool.close()
        pool.join()

        while not q.empty():
            ap_scores.append(q.get())

    try:
        mean_ap = sum(ap_scores)/len(ap_scores)
    except ZeroDivisionError:
        mean_ap = 0

    return mean_ap, mIOU

'\nimport multiprocessing as mp\n\ndef compute_confidences_ap(gt_boxes, N_gt, predicted_boxes, N=10, iou_th=0.5):\n    ap_scores = []\n\n    for i in range(N):\n        random.shuffle(predicted_boxes)\n        mIOU, _, ap = mean_AP_Pascal_VOC(gt_boxes, N_gt, predicted_boxes, iou_th)\n        ap_scores.append(ap)\n\n    try:\n        mean_ap = sum(ap_scores)/len(ap_scores)\n    except ZeroDivisionError:\n        mean_ap = 0\n\n    return mean_ap, mIOU\n\ndef compute_ap_score(q, gt_boxes, N_gt, predicted_boxes, iou_th):\n    random.shuffle(predicted_boxes)\n    mIOU, _, ap = mean_AP_Pascal_VOC(gt_boxes, N_gt, predicted_boxes, iou_th)\n    q.put(ap)\n\ndef parallel_compute_confidences_ap(gt_boxes, N_gt, predicted_boxes, N=10, iou_th=0.5):\n    ap_scores = []\n\n    with mp.Pool() as pool:\n        q = mp.Queue()\n        for i in range(N):\n            pool.apply_async(compute_ap_score, args=(q, gt_boxes, N_gt, predicted_boxes, iou_th))\n\n        pool.close()\n        pool.join()\n\n    

# TASK 1.1

In [21]:
gt_boxes,total = load_from_xml('../dataset/ai_challenge_s03_c010-full_annotation.xml')
df = pd.DataFrame(columns=['mean','std','delete_probability','generate_probability','mIOU','mAP'])
video_capture = cv2.VideoCapture('../dataset/AICity_data/train/S03/c010/vdo.avi')

NOISE PARAMETERS

In [22]:
mean = [-2,0,2]
std = [2,5,10,15]

for i in range(len(mean)):
    noisy_bboxes = generate_noisy_boxes(gt_boxes,del_prob=0.1,gen_prob=0.1,mean=mean[i],std=1)
    mAP,mIOU = compute_confidences_ap(gt_boxes,total,noisy_bboxes)
    df.at[i,'mean'] = mean[i]
    df.at[i,'std'] = 1
    df.at[i,'delete_probability'] = 0.1
    df.at[i,'generate_probability'] = 0.1
    df.at[i,'mIOU'] = mIOU
    df.at[i,'mAP'] = mAP


for i in range(len(std)):
    noisy_bboxes = generate_noisy_boxes(gt_boxes,del_prob=0.1,gen_prob=0.1,mean=0,std=std[i])
    mAP,mIOU = compute_confidences_ap(gt_boxes,total,noisy_bboxes)
    df.at[i+3,'mean'] = 0
    df.at[i+3,'std'] = std[i]
    df.at[i+3,'delete_probability'] = 0.1
    df.at[i+3,'generate_probability'] = 0.1
    df.at[i+3,'mIOU'] = mIOU
    df.at[i+3,'mAP'] = mAP


noisy_bboxes = generate_noisy_boxes(gt_boxes,del_prob=0.7,gen_prob=0.1,mean=0,std=1)
mAP,mIOU = compute_confidences_ap(gt_boxes,total,noisy_bboxes)
df.at[7,'mean'] = 0
df.at[7,'std'] = 1
df.at[7,'delete_probability'] = 0.7
df.at[7,'generate_probability'] = 0.1
df.at[7,'mIOU'] = mIOU
df.at[7,'mAP'] = mAP

noisy_bboxes = generate_noisy_boxes(gt_boxes,del_prob=0.1,gen_prob=0.7,mean=0,std=1)
mAP,mIOU = compute_confidences_ap(gt_boxes,total,noisy_bboxes)
df.at[8,'mean'] = 0
df.at[8,'std'] = 1
df.at[8,'delete_probability'] = 0.1
df.at[8,'generate_probability'] = 0.7
df.at[8,'mIOU'] = mIOU
df.at[8,'mAP'] = mAP


100%|██████████| 10/10 [06:15<00:00, 37.60s/it]


In [23]:
noisy_bboxes = generate_noisy_boxes(gt_boxes,del_prob=0.1,gen_prob=0.1,mean=-2,std=15)
mAP,mIOU = compute_confidences_ap(gt_boxes,total,noisy_bboxes)
df.at[9,'mean'] = -2
df.at[9,'std'] = 15
df.at[9,'delete_probability'] = 0.1
df.at[9,'generate_probability'] = 0.1
df.at[9,'mIOU'] = mIOU
df.at[9,'mAP'] = mAP

100%|██████████| 10/10 [05:52<00:00, 35.25s/it]


In [24]:
df

,mean,std,delete_probability,generate_probability,mIOU,mAP
0,-2,1,0.1,0.1,0.830711,0.810786
1,0,1,0.1,0.1,0.886393,0.902075
2,2,1,0.1,0.1,0.832345,0.899158
3,0,2,0.1,0.1,0.850911,0.90088
4,0,5,0.1,0.1,0.752073,0.808429
5,0,10,0.1,0.1,0.62828,0.630966
6,0,15,0.1,0.1,0.537554,0.367815
7,0,1,0.7,0.1,0.368466,0.355724
8,0,1,0.1,0.7,0.88211,0.767342
9,-2,15,0.1,0.1,0.533034,0.370982


### Bounding box visualization

In [14]:
def bounding_box_visualization(path,gt_boxes,predicted_boxes,video_capture,frame_id,iou_scores):
    n_frame = int(frame_id.split('_')[-1])
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, n_frame-1)
    res, frame = video_capture.read()
    # Draw the ground truth boxes
    for box in gt_boxes[frame_id]:
        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
    # Draw the predicted boxes
    for box in predicted_boxes[frame_id]:
        cv2.rectangle(frame, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (0, 0, 255), 2)
    # put text mIOU of frame
    cv2.putText(frame, f"IoU score: {iou_scores[n_frame]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    # put text number of frame
    cv2.putText(frame, f"Frame: {n_frame}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            

    cv2.imwrite(f'{path}/{frame_id}.png', frame)

    ret, frame = video_capture.read()
    

In [15]:
noisy_bboxes = generate_noisy_boxes(gt_boxes,0.6,0.1,0,25)
mIOU, mIOU_frame ,AP = mean_AP_Pascal_VOC(gt_boxes,total,noisy_bboxes,iou_th=0.5)
noisy_bboxes = group_by_frame(noisy_bboxes)

frames_id = list(noisy_bboxes.keys())
frames_id.sort(key=lambda x: int(x.split('_')[1]))
# frames = random.sample(frames_id, 5)
frames = ['f_175', 'f_767', 'f_1011', 'f_1081', 'f_1337']
iou_scores = [np.mean(mIOU_frame[frame]) for frame in frames_id]

path = '/home/guillem/mcv-m6-2023-team6/week1/plots/noisy_bboxes'
for frame in frames:
    bounding_box_visualization(path,gt_boxes,noisy_bboxes,video_capture,frame,iou_scores)
    print(f'{frame} IoU bboxes:')
    print(mIOU_frame[frame])

f_175 IoU bboxes:
[0.43456062520842975, 0.7108643069676075, 0.007875262447701505]
f_767 IoU bboxes:
[0.7139145001648474, 0.3032488545696072]
f_1011 IoU bboxes:
[0.007628701267247318, 0.5523287588859707, 0.4394843907260803, 0.5909938548739999]
f_1081 IoU bboxes:
[0.04670484684984797, 0.8496561292245814, 0.0073490222852221855, 0.40928530635311655, 0.6514520622502601, 0.6848043568032967]
f_1337 IoU bboxes:
[0.21918918859668926, 0.2181633922864014, 0.618954511080049, 0.466821999881477, 0.45354257116265817, 0.6722578115126581]


In [16]:
# generate_noisy_boxes(gt_boxes,del_prob=0.1,gen_prob=0.1,mean=mean[i],std=1)
def noisy_boxes_visualization(path,gt_boxes,video_capture, prob_del, prob_gen, mean, std):
    noisy_bboxes = generate_noisy_boxes(gt_boxes,prob_gen,prob_del,mean,std)
    mIOU, mIOU_frame ,AP = mean_AP_Pascal_VOC(gt_boxes,total,noisy_bboxes,iou_th=0.5)
    noisy_bboxes = group_by_frame(noisy_bboxes)

    frames_id = list(noisy_bboxes.keys())
    frames_id.sort(key=lambda x: int(x.split('_')[1]))
    # frames = random.sample(frames_id, 5)
    frames = ['f_175', 'f_767', 'f_1011', 'f_1081', 'f_1337']
    iou_scores = [np.mean(mIOU_frame[frame]) for frame in frames_id]


    for frame in frames:
        bounding_box_visualization(path,gt_boxes,noisy_bboxes,video_capture,frame,iou_scores)
        print(f'{frame} IoU bboxes:')
        print(mIOU_frame[frame])

noisy_boxes_visualization('/home/guillem/mcv-m6-2023-team6/week1/plots/noisy_bboxes/mean', gt_boxes, video_capture, 0, 0, 20, 0 )
noisy_boxes_visualization('/home/guillem/mcv-m6-2023-team6/week1/plots/noisy_bboxes/std', gt_boxes, video_capture, 0, 0, 0, 15 )
noisy_boxes_visualization('/home/guillem/mcv-m6-2023-team6/week1/plots/noisy_bboxes/delete', gt_boxes, video_capture, 0.7, 0, 0, 0 )
noisy_boxes_visualization('/home/guillem/mcv-m6-2023-team6/week1/plots/noisy_bboxes/generate', gt_boxes, video_capture, 0, 0.7, 0, 0 )

f_175 IoU bboxes:
[0.42611794652538043, 0.5350830248133291, 0.30081740144482094, 0.37653236935086426, 0.2274991408544769, 0.4122363254822492, 0.6872996886950804, 0.3667488643135726]
f_767 IoU bboxes:
[0.6736256591268304, 0.3412872543991964, 0.42611794652538043, 0.5350830248133291, 0.30081740144482094, 0.37653236935086426, 0.2274991408544769, 0.4122363254822492, 0.6872996886950804, 0.3667488643135726]
f_1011 IoU bboxes:
[0.29688992569204936, 0.42611794652538043, 0.5350830248133291, 0.30081740144482094, 0.37653236935086426, 0.2274991408544769, 0.4122363254822492, 0.6872996886950804, 0.3667488643135726]
f_1081 IoU bboxes:
[0.2592439119317871, 0.28785011576200575, 0.42611794652538043, 0.5350830248133291, 0.30081740144482094, 0.37653236935086426, 0.2274991408544769, 0.4122363254822492, 0.6872996886950804, 0.3667488643135726]
f_1337 IoU bboxes:
[0.30955381094307527, 0.267051255717225, 0.42611794652538043, 0.5350830248133291, 0.30081740144482094, 0.37653236935086426, 0.2274991408544769, 0.412

KeyError: 'f_767'

# TASK 1.2

● Compute the mAP for the provided detections (mask_rcnn, ssd512, yolo3)

In [ ]:
maskrcnn = '../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt'
ssd = '../dataset/AICity_data/train/S03/c010/det/det_ssd512.txt'
yolo = '../dataset/AICity_data/train/S03/c010/det/det_yolo3.txt' 

In [ ]:
maskrcnn = '../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt'
ssd = '../dataset/AICity_data/train/S03/c010/det/det_ssd512.txt'
yolo = '../dataset/AICity_data/train/S03/c010/det/det_yolo3.txt' 

paths = [maskrcnn,yolo,ssd]
for path in paths:
    boxes = load_from_txt(path)
    # Sort by confidence
    boxes.sort(key=lambda x:x[-1], reverse=True)
    mIOU, _ ,AP = mean_AP_Pascal_VOC(gt_boxes,total,boxes,iou_th=0.5)

    print(path.split('/')[-1],':')
    print(mIOU, AP)

# TASK 2


In [ ]:

paths = ['noisy', maskrcnn,yolo,ssd]
name = ['Noisy', 'Mask R-CNN','YOLOv3','SSD512']
paths = [maskrcnn,yolo,ssd]
name = ['Mask R-CNN','YOLOv3','SSD512']
for path,name in zip(paths, name):
    if path == 'noisy':
        boxes = generate_noisy_boxes(gt_boxes,0.6,0.1,0,10)
    else:
        boxes = load_from_txt(path)
    
    boxes.sort(key=lambda x:x[-1], reverse=True)
    # mIOU, mIOU_frame ,AP = mean_AP_Pascal_VOC(gt_boxes,total,boxes,iou_th=0.5)
    mIOU, mIOU_frame = mean_IoU_nonrestricted(gt_boxes,boxes)

    print(f'Mean IoU {name}: {mIOU}')
    # print(f'Mean AP {name}: {AP}')

    # Plot the IoU score for each frame
    fig = plt.figure(figsize=(10,5))
    # Set title
    plt.title(name)
    frames = list(mIOU_frame.keys())
    # Sort the frames
    frames.sort(key=lambda x: int(x.split('_')[1]))
    # Convert frames to a list of integers
    frames_num = [int(frame.split('_')[1]) for frame in frames]
    iou_scores = [np.mean(mIOU_frame[frame]) for frame in frames]
    plt.plot(frames_num, iou_scores)
    plt.xlabel('Frame')
    plt.ylabel('IoU score')
    # y from 0 to 1
    plt.ylim(0,1)
    plt.show()



In [ ]:
"""
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def group_by_frame(predicted_boxes):
    predicted_boxes.sort(key=lambda x: x[0])
    predicted_boxes = itertools.groupby(predicted_boxes, key=lambda x: x[0])
    predicted_boxes = {k: list(v) for k, v in predicted_boxes}
    return predicted_boxes

def create_animation(path, gt_boxes, predicted_boxes, video_capture):
    # Group the detected boxes by frame_id as a dictionary
    predicted_boxes_group = group_by_frame(predicted_boxes)
    # Get the IoU score for each frame in format dict {frame_id: [iou_score1, iou_score2, ...]}
    mIOU, mIOU_frame ,AP = mean_AP_Pascal_VOC(gt_boxes,total,predicted_boxes,iou_th=0.5)
    # Get the frame_id list
    frames_id = list(mIOU_frame.keys())
    # Sort the frames list
    frames_id.sort(key=lambda x: int(x.split('_')[1]))
    frames_num = [int(frame.split('_')[1]) for frame in frames_id]
    # Get the IoU score list
    iou_scores = [np.mean(mIOU_frame[frame]) for frame in frames_id]
    # Get the video fps
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    # Get the video width
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    # Get the video height
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a figure
    fig = plt.figure(figsize=(10, 10))

    # Create a subplot for the video frames
    ax1 = fig.add_subplot(211)
    # Hide the x and y ticks and labels
    ax1.set_xticks([])
    ax1.set_yticks([])

    # Create a subplot for the IoU line plot
    ax2 = fig.add_subplot(212)
    # Set the title
    ax2.set_title('IoU score for each frame')
    # Set the x label
    ax2.set_xlabel('Frame')
    # Set the y label
    ax2.set_ylabel('IoU score')
    # Set the x axis range
    ax2.set_xlim(0, frames_num[-1])
    # Set the y axis range
    ax2.set_ylim(0, 1)
    # Create a line
    line, = ax2.plot([], [], lw=2)

    print(gt_boxes[frames_id[0]])
    print(predicted_boxes_group[frames_id[0]])

    # Create a function to initialize the animation
    def init():
        line.set_data([], [])
        # text.set_text('')
        return line,

    # Create a function to update the animation
    def animate(i):
        # Update IoU line plot
        line.set_data(frames_num[:i], iou_scores[:i])

        # # Update video frames
        ret, frame = video_capture.read()
        if ret:
            # Convert the frame to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Draw the ground truth boxes
            for box in gt_boxes[frames_id[i]]:
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            # Draw the predicted boxes
            for box in predicted_boxes_group[frames_id[i]]:
                cv2.rectangle(frame, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (0, 0, 255), 2)
            # Set the frame to the subplot in rgb format
            ax1.imshow(frame)

        return line


    # # Create the animation
    ani = animation.FuncAnimation(
        fig, animate, frames=None, init_func=init)#, interval=1000/fps, blit=True, repeat=False)

    ani.save(f'Prova.mp4', fps=fps, bitrate=-1,
            extra_args=['-vcodec', 'libx264'])
"""
gt_boxes,total = load_from_xml('../dataset/ai_challenge_s03_c010-full_annotation.xml')
predicted_boxes = load_from_txt('../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt')
predicted_boxes.sort(key=lambda x:x[-1], reverse=True)
video_capture = cv2.VideoCapture('../dataset/AICity_data/train/S03/c010/vdo.avi')
# create_animation('../dataset/AICity_data/train/S03/c010/vdo.avi', gt_boxes, predicted_boxes, video_capture)



In [ ]:
gt_boxes,total = load_from_xml('../dataset/ai_challenge_s03_c010-full_annotation.xml')
predicted_boxes = load_from_txt('../dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt')
predicted_boxes.sort(key=lambda x:x[-1], reverse=True)
video_capture = cv2.VideoCapture('../dataset/AICity_data/train/S03/c010/vdo.avi')

In [ ]:
import itertools
import numpy as np
import cv2

def group_by_frame(predicted_boxes):
    """Group the detected boxes by frame_id as a dictionary"""
    predicted_boxes.sort(key=lambda x: x[0])
    predicted_boxes = itertools.groupby(predicted_boxes, key=lambda x: x[0])
    predicted_boxes = {k: list(v) for k, v in predicted_boxes}
    return predicted_boxes

def create_video(path, gt_boxes, predicted_boxes, video_capture):
    """Create a video with the IoU score for each frame"""
    # Group the detected boxes by frame_id as a dictionary
    predicted_boxes_group = group_by_frame(predicted_boxes)
    # Get the IoU score for each frame in format dict {frame_id: [iou_score1, iou_score2, ...]}
    mIOU, mIOU_frame ,AP = mean_AP_Pascal_VOC(gt_boxes,total,predicted_boxes,iou_th=0.5)
    # Get the frame_id list
    frames_id = list(mIOU_frame.keys())
    # Sort the frames list
    frames_id.sort(key=lambda x: int(x.split('_')[1]))
    frames_num = [int(frame.split('_')[1]) for frame in frames_id]
    # Get the IoU score list
    iou_scores = [np.mean(mIOU_frame[frame]) for frame in frames_id]
    # Get the video fps
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    # Get the video width
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    # Get the video height
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(path, fourcc, fps, (width, height))

    # Loop through each frame
    for i, frame_id in enumerate(frames_id):
        # Read the frame
        ret, frame = video_capture.read()
        if ret:
            # Convert the frame to RGB
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Draw the ground truth boxes
            for box in gt_boxes[frame_id]:
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            # Draw the predicted boxes
            for box in predicted_boxes_group[frame_id]:
                cv2.rectangle(frame, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (0, 0, 255), 2)
            # Draw the IoU score
            iou_score = round(iou_scores[i], 2)
            cv2.putText(frame, f"IoU score: {iou_score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # put text number of frame
            cv2.putText(frame, f"Frame: {frames_num[i]}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # put fps
            cv2.putText(frame, f"FPS: {fps}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # Write the frame to the video
            out.write(frame)

    # Release the VideoWriter object
    out.release()




In [ ]:
create_video('Prova.mp4', gt_boxes, predicted_boxes, video_capture)